# Complete Data and Data by Year Heatmap Creation

In [ ]:
import folium
from folium import plugins
from folium.plugins import HeatMap
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.simplefilter("ignore")

In [186]:
df = pd.read_csv('C:/Users/Henry/Documents/UVA/CS5010/Final_Project/Parking_Tickets.csv')
df['newlocation'] = df.Location + ' Charlottesville, VA'
geocoded = pd.read_csv('C:/Users/Henry/Documents/UVA/CS5010/Final_Project/outputgeocoder.csv')
geocoded = geocoded[['input_string','latitude','longitude']]
merged = pd.merge(df, geocoded, left_on = 'newlocation', right_on = 'input_string')
missing = merged[merged.latitude.isnull()] 
print(missing.Location.value_counts()) #show missing locations
mergedfull = merged[pd.notnull(merged.latitude)]
latlong = mergedfull[['latitude','longitude']]

424 TOW AWAY ZONE    1
Name: Location, dtype: int64


In [168]:
m = folium.Map([38.0345,-78.49], zoom_start = 14)

In [169]:
heat_data = [[row['latitude'],row['longitude']]for index, row in latlong.iterrows()]
HeatMap(heat_data, min_opacity = .1,max_zoom =18, max_val = 1, blur = 2,radius = 4).add_to(m)

In [170]:
m.save('ParkingTicketsFULLDATAHEATONLY.html')

In [187]:
mergedfull['Date'] = mergedfull.DateIssued.str[:10]
mergedfull['Time'] = mergedfull.DateIssued.str[11:-5]
mergedfull['Hour'] = pd.to_datetime(mergedfull['Time'], format= '%H:%M:%S' ).dt.hour
mergedfull['Hour'] = mergedfull.Hour.map("{:02}".format)
mergedfull = mergedfull.drop(columns=['RecordID','TicketNumber','DateIssued', 'StreetName','StreetNumber','LicenseState','WaiverRequestDate','WaiverGrantedDate','AppealDate','AppealGrantedDate','ViolationDescription','AppealStatus','Location','LicensePlateAnon'])
mergedfull['Hour'] = np.where(mergedfull.eval("Time == '00:00:00'"), mergedfull.TimeIssued.str[:2], mergedfull.Hour)
mergedfull['Date'] = pd.to_datetime(mergedfull['Date'], infer_datetime_format = True, errors='coerce')
date_current = pd.to_datetime(datetime.date.today())
current = mergedfull[mergedfull['Date'] < date_current]
current['Hour'] = pd.to_numeric(current['Hour'])
current = current[current['Hour'] < 24]
current['Year'] = current['Date'].dt.year

In [183]:
for x in range(2000,2019):
    yeardf = current[current.Year == x]
    yeardf = yeardf[['latitude','longitude']]
    m = folium.Map([38.0345,-78.49], zoom_start = 14)
    heat_data = [[row['latitude'],row['longitude']]for index, row in yeardf.iterrows()]
    HeatMap(heat_data, min_opacity = .4,max_zoom =18, max_val = 1, blur = 3,radius = 5).add_to(m)
    m.save('ParkingTickets'+str(x)+'.html')